In [2]:
import re

def remove_comments(sql):
    # Loại bỏ các comment kiểu -- đến cuối dòng
    sql = re.sub(r'--.*', '', sql)
    # Loại bỏ các comment kiểu /* ... */
    sql = re.sub(r'/\*.*?\*/', '', sql, flags=re.DOTALL)
    return sql

def extract_jobs_and_sql(dsx_content):
    # Tìm tất cả các tên job trong DSJOB
    job_names = re.findall(r'BEGIN DSJOB.*?Identifier "(.*?)"', dsx_content, re.DOTALL)

    # Tìm tất cả các đoạn CDATA chứa câu truy vấn SQL
    sql_queries = re.findall(r'<!\[CDATA\[(.*?)\]\]>', dsx_content, re.DOTALL | re.IGNORECASE)

    cleaned_sql_queries = []
    for query in sql_queries:
        # Loại bỏ khoảng trắng thừa ở đầu và cuối chuỗi
        query = query.strip()

        # Loại bỏ comment trong câu truy vấn
        query = remove_comments(query)
        
        # Chỉ lấy những câu truy vấn bắt đầu bằng từ khóa SQL chính xác (ví dụ: SELECT, UPDATE, INSERT INTO, DELETE FROM)
        sql_keywords = ['SELECT', 'UPDATE', 'INSERT INTO', 'DELETE FROM']
        for keyword in sql_keywords:
            if query.upper().startswith(keyword):
                # Loại bỏ phần trước từ khóa và chỉ giữ lại câu truy vấn
                cleaned_query = re.sub(rf'^.*?\b{keyword}\b', keyword, query, flags=re.DOTALL)
                # Cắt bỏ bất kỳ phần nào sau dấu chấm phẩy đầu tiên
                cleaned_query = re.split(r';', cleaned_query)[0]

                # Thêm dấu chấm phẩy vào cuối câu nếu chưa có
                if not cleaned_query.endswith(';'):
                    cleaned_query += ';'

                cleaned_sql_queries.append(cleaned_query)
                break

    return job_names, cleaned_sql_queries

# Đọc nội dung của file .dsx
with open('FNC_SVC_AR_FCT_FA.dsx', 'r', encoding='utf-8') as file:
    dsx_content = file.read()

# Gọi hàm để trích xuất tên job và các truy vấn SQL
job_names, sql_queries = extract_jobs_and_sql(dsx_content)

# Hiển thị các job và các truy vấn SQL
print("Job Names:")
for job in job_names:
    print(job)

print("\nSQL Queries:")
for query in sql_queries:
    print(query)


Job Names:
PAR_DMT_FNC_SVC_AR_ANL_FCT_FA_Ins
PAR_DMT_TWT_FNC_SVC_AR_ANL_FCT_FA_CHG_TruncIns
PAR_DMT_TWT_FNC_SVC_AR_ANL_FCT_FA_CST_PD_TruncIns
PAR_DMT_TWT_FNC_SVC_AR_ANL_FCT_FA_LST_DY_TruncIns
PAR_DMT_TWT_FNC_SVC_AR_ANL_FCT_FA_TruncIns
PAR_DMT_TWT_FNC_SVC_AR_FCT_FA_TruncIns

SQL Queries:
SELECT
	AC_AR_DIM.AC_AR_DIM_ID					AC_AR_DIM_ID
	,CST_DIM.CST_DIM_ID						CST_DIM_ID
	,LOAN_PPS_DIM.LOAN_PPS_DIM_ID			LOAN_PPS_DIM_ID
	,PD_DIM.PD_DIM_ID						PD_DIM_ID
	,RLTNP_OFCR.EMPE_DIM_ID					RLTNP_OFCR_DIM_ID
	,IDY_DIM.IDY_DIM_ID						IDY_DIM_ID
	,LDGR_ACG_DIM.LDGR_ACG_DIM_ID			LDGR_ACG_DIM_ID
	,TERM_DIM.GRP_DIM_ID					TERM_DIM_ID
	,ADJ_OU.OU_DIM_ID						ADJ_OU_DIM_ID
	,ADJ_CCY.CCY_DIM_ID						ADJ_CCY_DIM_ID
	,ADJ_LOB.LOB_DIM_ID						ADJ_LOB_DIM_ID
	,CST_FNC_ST_DIM.FNC_ST_DIM_ID			CST_FNC_ST_DIM_ID
	,ADJ_FNC_ST_DIM.FNC_ST_DIM_ID			ADJ_FNC_ST_DIM_ID
	,MIS_ADJ_FNC_ST_DIM.FNC_ST_DIM_ID		MIS_ADJ_FNC_ST_DIM_ID
	,FNL_FNC_ST_DIM.FNC_ST_DIM_ID			FNL_FNC_ST_DIM_ID
	,FNC_ST_TP_DIM.GRP_DIM_ID				FNC_ST_TP_DI

In [24]:
import re
import pandas as pd
from pprint import pprint

def remove_comments(sql):
    # Loại bỏ các comment kiểu -- từ dấu -- cho đến khi gặp dấu xuống dòng \n
    sql = re.sub(r'--.*?\n', '', sql)
    # Loại bỏ các comment kiểu /* ... */
    sql = re.sub(r'/\*.*?\*/', '', sql, flags=re.DOTALL)
    return sql

def extract_jobs_and_sql(dsx_content):
    # Tìm tất cả các job với tên và vị trí bắt đầu
    job_matches = re.finditer(r'BEGIN DSJOB.*?Identifier "(.*?)"', dsx_content, re.DOTALL)
    jobs = [(match.start(), match.group(1)) for match in job_matches]

    # Tìm tất cả các đoạn CDATA chứa câu truy vấn SQL với vị trí của chúng
    sql_matches = re.finditer(r'<!\[CDATA\[(.*?)\]\]>', dsx_content, re.DOTALL | re.IGNORECASE)
    
    sql_queries = [(match.start(), match.group(1).strip()) for match in sql_matches]
    pprint(sql_queries)
    job_queries = []
    
    # Duyệt qua từng câu truy vấn và gán nó vào job tương ứng
    for sql_pos, query in sql_queries:
        # Loại bỏ comment trong câu truy vấn
        query = remove_comments(query)
        
        # Chỉ lấy những câu truy vấn bắt đầu bằng từ khóa SQL chính xác
        sql_keywords = ['SELECT', 'UPDATE', 'INSERT INTO', 'DELETE FROM', 'WITH']
        for keyword in sql_keywords:
            if query.upper().startswith(keyword):
                # Loại bỏ phần trước từ khóa và chỉ giữ lại câu truy vấn
                cleaned_query = re.sub(rf'^.*?\b{keyword}\b', keyword, query, flags=re.DOTALL | re.IGNORECASE)
                # Cắt bỏ bất kỳ phần nào sau dấu chấm phẩy đầu tiên
                cleaned_query = re.split(r';', cleaned_query)[0]
                
                # Thêm dấu chấm phẩy vào cuối câu nếu chưa có
                if not cleaned_query.endswith(';'):
                    cleaned_query += ';'

                # Xác định job chứa câu truy vấn này
                job_name = None
                for i in range(len(jobs) - 1):
                    if jobs[i][0] <= sql_pos < jobs[i + 1][0]:
                        job_name = jobs[i][1]
                        break
                if job_name is None and jobs:
                    job_name = jobs[-1][1]

                job_queries.append((job_name, cleaned_query))
                break

    return job_queries

# Đọc nội dung của file .dsx
with open('FNC_SVC_AR_FCT_FA.dsx', 'r', encoding='utf-8') as file:
    dsx_content = file.read()

# Gọi hàm để trích xuất tên job và các truy vấn SQL
job_queries = extract_jobs_and_sql(dsx_content)

# Hiển thị các job và các truy vấn SQL tương ứng
for job, query in job_queries:
    print(f"Job Name: {job}")
    #print(f"SQL Query: {query}\n")

# Chuyển đổi job_queries thành DataFrame
df = pd.DataFrame(job_queries, columns=["Job Name", "SQL Query"])

# Xuất DataFrame ra file Excel
df.to_excel("job_queries.xlsx", index=False)

print("Data has been exported to job_queries.xlsx")

[(7259, '#PS_ORA_CONN_DMT.ConnectionString#'),
 (7351, '#PS_ORA_CONN_DMT.Username#'),
 (7437, '#PS_ORA_CONN_DMT.Password#'),
 (7534, '0'),
 (7594, '11g'),
 (7660, '0'),
 (7709, '0'),
 (7765, '0'),
 (7845,
  'SELECT\n'
  '\tAC_AR_DIM.AC_AR_DIM_ID\t\t\t\t\tAC_AR_DIM_ID\n'
  '\t,CST_DIM.CST_DIM_ID\t\t\t\t\t\tCST_DIM_ID\n'
  '\t,LOAN_PPS_DIM.LOAN_PPS_DIM_ID\t\t\tLOAN_PPS_DIM_ID\n'
  '\t,PD_DIM.PD_DIM_ID\t\t\t\t\t\tPD_DIM_ID\n'
  '\t,RLTNP_OFCR.EMPE_DIM_ID\t\t\t\t\tRLTNP_OFCR_DIM_ID\n'
  '\t,IDY_DIM.IDY_DIM_ID\t\t\t\t\t\tIDY_DIM_ID\n'
  '\t,LDGR_ACG_DIM.LDGR_ACG_DIM_ID\t\t\tLDGR_ACG_DIM_ID\n'
  '\t,TERM_DIM.GRP_DIM_ID\t\t\t\t\tTERM_DIM_ID\n'
  '\t,ADJ_OU.OU_DIM_ID\t\t\t\t\t\tADJ_OU_DIM_ID\n'
  '\t,ADJ_CCY.CCY_DIM_ID\t\t\t\t\t\tADJ_CCY_DIM_ID\n'
  '\t,ADJ_LOB.LOB_DIM_ID\t\t\t\t\t\tADJ_LOB_DIM_ID\n'
  '\t,CST_FNC_ST_DIM.FNC_ST_DIM_ID\t\t\tCST_FNC_ST_DIM_ID\n'
  '\t,ADJ_FNC_ST_DIM.FNC_ST_DIM_ID\t\t\tADJ_FNC_ST_DIM_ID\n'
  '\t,MIS_ADJ_FNC_ST_DIM.FNC_ST_DIM_ID\t\tMIS_ADJ_FNC_ST_DIM_ID\n'
  '\t,F

In [8]:
b = """
select
chg.AC_AR_ID
,decode(CHG.MUD, 'OD', ABS(CLS_VAL_BAL_FCY*CHG.ADJ_BAL), CLS_VAL_BAL_FCY*CHG.ADJ_BAL)	CLS_BAL_FCY   
,decode(CHG.MUD, 'OD', ABS(CLS_VAL_BAL_LCY*CHG.ADJ_BAL), CLS_VAL_BAL_LCY*CHG.ADJ_BAL)	CLS_BAL_LCY
,TVR.LQD_AMT_FCY                               			LQD_AMT_FCY
,TVR.LQD_AMT_LCY                       	    			LQD_AMT_LCY
,TVR.DSBR_AMT_TDY_FCY                                   DSBR_AMT_FCY
,TVR.DSBR_AMT_TDY                                      	DSBR_AMT_LCY
,TVR.PREPYMT_AMT	                                    PREPYMT_AMT_FCY
,TVR.PREPYMT_AMT*TVR.EXG_RATE_TO_LCL_CCY	            PREPYMT_AMT_LCY
,TVR.PY_OFF_BFR_DUE_DT_F                              	PY_OFF_BFR_DUE_DT_F
,decode(tvr.CLS_VAL_BAL_FCY*CHG.ADJ_BAL , 0, 'NORM', 
		COALESCE(CAST(CHG.LN_FNC_ST_CODE AS VARCHAR2(100)), ARF.CL_CODE, CAST(CHG.CARD_AR_FNC_ST_TP_CODE AS VARCHAR2(100))))	    ADJ_FNC_ST_CODE --NHOM NO
,CASE WHEN decode(tvr.CLS_VAL_BAL_FCY*CHG.ADJ_BAL , 0, 'NORM', 
		COALESCE(CAST(CHG.LN_FNC_ST_CODE AS VARCHAR2(100)), ARF.CL_CODE, CAST(CHG.CARD_AR_FNC_ST_TP_CODE AS VARCHAR2(100)))) <> 'NORM' AND PNP_ARS + INT_ARS > 0 THEN 'THUC_TE' 
      WHEN decode(tvr.CLS_VAL_BAL_FCY*CHG.ADJ_BAL , 0, 'NORM', 
		COALESCE(CAST(CHG.LN_FNC_ST_CODE AS VARCHAR2(100)), ARF.CL_CODE, CAST(CHG.CARD_AR_FNC_ST_TP_CODE AS VARCHAR2(100)))) <> 'NORM' AND PNP_ARS + INT_ARS = 0 THEN 'THU_THACH'
 END                                        			FNC_ST_TP_CODE
-- Add new luc.luu
,DLQ.PNP_ARS
,DLQ.INT_ARS
,DLQ.NOD_PAST_DUE
,TVR.DSBR_AMT_LTD TOT_DSBR_AMT_FCY
,TVR.INTERIM_INT
,CASE WHEN DLQ.PNP_PAST_DUE_DT <= nvl(DLQ.INT_PAST_DUE_DT,99991231) AND PAST_DUE_F = 1
	  AND  DLQ.PNP_PAST_DUE_DT IS NOT NULL 
	  THEN MONTHS_BETWEEN(Trunc(to_date(#pdate#,'yyyymmdd'),'MONTH'),Trunc(to_date(DLQ.PNP_PAST_DUE_DT,'yyyymmdd'),'MONTH')) 
	  WHEN PAST_DUE_F = 1 AND DLQ.INT_PAST_DUE_DT <= nvl(DLQ.PNP_PAST_DUE_DT,99991231) 
	  AND  DLQ.INT_PAST_DUE_DT IS NOT NULL 
	  THEN MONTHS_BETWEEN(Trunc(to_date(#pdate#,'yyyymmdd'),'MONTH'),Trunc(to_date(DLQ.INT_PAST_DUE_DT,'yyyymmdd'),'MONTH'))
	  ELSE 0 END DLQ_CNT
,decode(CHG.MUD, 'OD', ABS(CLS_VAL_BAL_FCY), CLS_VAL_BAL_FCY) ORIG_CLS_BAL_FCY
,decode(CHG.MUD, 'OD', ABS(CLS_VAL_BAL_LCY), CLS_VAL_BAL_LCY) ORIG_CLS_BAL_LCY
FROM TWT_FNC_SVC_AR_ANL_FCT_FA_CHG chg
LEFT JOIN AR_DLQ_SMY DLQ ON ac_AR_ID = DLQ.AR_ID AND DLQ.MSR_PRD_ID = #pDate#
LEFT JOIN TWT_AR_TVR_SMY_DAILY TVR ON ac_AR_ID = TVR.AR_ID AND TVR.MSR_PRD_ID = #pDate#
LEFT JOIN (SELECT ARF.AR_ID, ARF.EFF_DT, ARF.END_DT, CD.CL_CODE
	FROM AR_FNC_ST ARF
	LEFT JOIN CV CD ON CD.CL_ID = ARF.AR_FNC_ST_TP_ID
      WHERE ARF.AR_FNC_ST_SCM_ID = 'FF2F2563FA9BFED792EDC3A35EFDEE92') ARF ON AC_AR_ID = ARF.AR_ID 
	  AND ARF.EFF_DT <= LAST_EFF_DT AND ARF.END_DT > LAST_EFF_DT --LU_CV: 'AR_FNC_ST_SCM', 'USR_DFND_ST']"""
a = remove_comments(b)
a

"\nselect\nchg.AC_AR_ID\n,decode(CHG.MUD, 'OD', ABS(CLS_VAL_BAL_FCY*CHG.ADJ_BAL), CLS_VAL_BAL_FCY*CHG.ADJ_BAL)\tCLS_BAL_FCY   \n,decode(CHG.MUD, 'OD', ABS(CLS_VAL_BAL_LCY*CHG.ADJ_BAL), CLS_VAL_BAL_LCY*CHG.ADJ_BAL)\tCLS_BAL_LCY\n,TVR.LQD_AMT_FCY                               \t\t\tLQD_AMT_FCY\n,TVR.LQD_AMT_LCY                       \t    \t\t\tLQD_AMT_LCY\n,TVR.DSBR_AMT_TDY_FCY                                   DSBR_AMT_FCY\n,TVR.DSBR_AMT_TDY                                      \tDSBR_AMT_LCY\n,TVR.PREPYMT_AMT\t                                    PREPYMT_AMT_FCY\n,TVR.PREPYMT_AMT*TVR.EXG_RATE_TO_LCL_CCY\t            PREPYMT_AMT_LCY\n,TVR.PY_OFF_BFR_DUE_DT_F                              \tPY_OFF_BFR_DUE_DT_F\n,decode(tvr.CLS_VAL_BAL_FCY*CHG.ADJ_BAL , 0, 'NORM', \n\t\tCOALESCE(CAST(CHG.LN_FNC_ST_CODE AS VARCHAR2(100)), ARF.CL_CODE, CAST(CHG.CARD_AR_FNC_ST_TP_CODE AS VARCHAR2(100))))\t    ADJ_FNC_ST_CODE ,CASE WHEN decode(tvr.CLS_VAL_BAL_FCY*CHG.ADJ_BAL , 0, 'NORM', \n\t\tCOALESCE(CAST(CH